In [1]:
# 1. Imports
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show

import pandas as pd
import numpy as np
# Standard library imports
import re

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    roc_auc_score,
    recall_score,
    balanced_accuracy_score,
)
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.utils import Bunch


In [2]:
# Define the chunk size for reading the CSV file
chunksize = 100000  # Adjust this value based on your requirements

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Read the CSV file in chunks based on the defined chunk size
for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
    # Filter the current chunk based on the criteria
    filtered_chunk = chunk[chunk['issue_d'].str.contains("2010", na=False)]
    # filtered_chunk = filtered_chunk[~filtered_chunk['issue_d'].str.contains("Oct-2013|Nov-2013|Dec-2013", na=False)]
    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks)

/tmp/ipykernel_12185/3547039439.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12185/3547039439.py:8: DtypeWarning: Columns (19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12185/3547039439.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_12185/3547039439.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Dat

In [3]:
import re

def preprocess_data_lc(df):
    # Calculate risk score
    # df["risk_score"] = df.loc[:, ["last_fico_range_high", "last_fico_range_low"]].mean(axis=1)
    
    # Create target variable
    df["target"] = np.where((df.loan_status == 'Current') |
                            (df.loan_status == 'Fully Paid') |
                            (df.loan_status == "Issued") |
                            (df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 0, 1)
    
    # Drop unnecessary columns
    # for c in ['last_fico_range_high', 'last_fico_range_low', 'loan_status']:
    for c in ['loan_status']:
        df = df.drop(c, axis=1, errors='ignore')
    
    def fix_term_column(df):
        """
        Fix the dtype of the 'term' column by extracting the numeric part and converting it to an integer.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'term' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'term' column converted to numeric.
        """
        try:
            # Extract numeric part from the 'term' column and convert to integer
            df['term'] = df['term'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notna(x) else x)
        except AttributeError as e:
            # Handle cases where the regex search fails (e.g., no match found)
            print("Error: Unable to extract numeric value from 'term' column.")
            print(f"Details: {e}")
        except Exception as e:
            # Handle any other unexpected errors
            print("An unexpected error occurred while fixing the 'term' column.")
            print(f"Details: {e}")
        
        return df
    
    def fix_emp_length_column(df):
        """
        Fix the dtype of the 'emp_length' column by converting it to numeric.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'emp_length' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'emp_length' column converted to numeric.
        """
        try:
            # Map '< 1 year' to '0' and extract numeric part from the 'emp_length' column
            df['emp_length'] = df['emp_length'].map(lambda x: "0" if x == '< 1 year' else x)
            df['emp_length'] = df['emp_length'].map(lambda x: int(re.search(r'\d+', str(x)).group()), na_action='ignore')
        except Exception as e:
            # Handle any unexpected errors
            print("Error: Unable to convert 'emp_length' column to numeric.")
            print(f"Details: {e}")
        
        return df
    
    df = fix_term_column(df)
    df = fix_emp_length_column(df)

    return df

In [4]:
df_lc_1 = preprocess_data_lc(df)
df_lc_1.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,target
1633373,643218,NaN,20000.0,20000.0,16358.045925,60,9.99,424.85,B,B4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1633396,642872,NaN,7475.0,7475.0,7475.000000,60,13.72,172.85,C,C5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1633407,642861,NaN,5575.0,5575.0,5575.000000,36,15.20,193.81,D,D4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1633409,642859,NaN,2150.0,2150.0,2150.000000,60,14.83,50.96,D,D3,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1633411,642857,NaN,7050.0,7050.0,7050.000000,60,12.98,160.34,C,C3,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
drop_list = ['member_id',
    'issue_d', 'url', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d', 'pymnt_plan', 'recoveries', 
    'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'hardship_flag', 
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 
    'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 
    'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 
    'settlement_percentage', 'settlement_term', 'id', 'emp_title', 'desc', 'title', 'last_credit_pull_d', 'next_pymnt_d',
]

In [7]:
X = df_lc_1.loc[:, df_lc_1.columns != 'target'].drop(drop_list, axis=1)#, 'risk_score'
y = df_lc_1['target']

df_2010 = pd.concat([X,y], axis = 1)
df_2010.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,target
1633373,20000.0,20000.0,16358.045925,60,9.99,424.85,B,B4,7.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
1633396,7475.0,7475.0,7475.000000,60,13.72,172.85,C,C5,0.0,OWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
1633407,5575.0,5575.0,5575.000000,36,15.20,193.81,D,D4,0.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
1633409,2150.0,2150.0,2150.000000,60,14.83,50.96,D,D3,0.0,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0
1633411,7050.0,7050.0,7050.000000,60,12.98,160.34,C,C3,0.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,0


In [25]:
import os

# Filter columns with less than 90% missing values
df_2010 = df_2010.loc[:, df_2010.isna().mean() < 0.9]

# Define path and ensure the directory exists
filepath = '/home/athyrson/Code/Data/Processed Data/Lending Club/df_2010.csv'
os.makedirs(os.path.dirname(filepath), exist_ok=True)

# Save to CSV
df_2010.to_csv(filepath, index=False)


12537

In [6]:
X = df_lc_1.loc[:, df_lc_1.columns != 'target'].drop(drop_list, axis=1)#, 'risk_score'
y = df_lc_1['target']

X_train_lc, X_test_lc, y_train_lc, y_test_lc = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_lc.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
1639041,12000.0,12000.0,11382.670844,36,7.51,373.33,A,A4,0.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1637548,4000.0,4000.0,4000.000000,60,15.95,97.17,D,D4,7.0,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1642419,5600.0,5600.0,5100.000000,36,6.76,172.31,A,A2,10.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1633912,25000.0,25000.0,23950.000000,60,14.83,592.52,D,D3,10.0,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1644346,3000.0,3000.0,3000.000000,36,10.25,97.16,B,B2,4.0,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [7]:
X_train = X_train_lc.copy()
X_test = X_test_lc.copy()
y_train, y_test = y_train_lc.copy(), y_test_lc.copy()

In [8]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score, mean_absolute_error

from interpret.glassbox import ExplainableBoostingRegressor

# Lista de modelos complexos
modelos = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "SVR": SVR(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000),
}

# Função utilitária para pipeline
def make_pipeline_model(model):
    return make_pipeline(TargetEncoder(), StandardScaler(), model)

# Lista para armazenar resultados
resultados = []

# Loop pelos modelos
for nome, modelo in modelos.items():
    pipeline_model = make_pipeline_model(modelo)
    pipeline_model.fit(X_train, y_train)
    y_pred_model = pipeline_model.predict(X_train)

    ebm = ExplainableBoostingRegressor(random_state=42)
    ebm.fit(X_train, y_pred_model)
    y_pred_ebm = ebm.predict(X_train)

    r2 = r2_score(y_pred_model, y_pred_ebm)
    mae = mean_absolute_error(y_pred_model, y_pred_ebm)

    resultados.append({
        "Modelo": nome,
        "R2_surrogate": round(r2, 4),
        "MAE_surrogate": round(mae, 4),
    })

# Criando DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)


/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:358: RuntimeWarning: All-NaN slice encountered
  min_feature_val = np.nanmin(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:359: RuntimeWarning: All-NaN slice encountered
  max_feature_val = np.nanmax(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/glassbox/_ebm/_ebm.py:813: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:358: RuntimeWarning: All-NaN slice encountered
  min_feature_val = np.nanmin(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-pack

          Modelo  R2_surrogate  MAE_surrogate
0  Random Forest        0.8799         0.0436
1        XGBoost        0.7231         0.0769
2            SVR        0.8625         0.0327
3  MLP Regressor        0.6963         0.0861


In [9]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from interpret.glassbox import ExplainableBoostingRegressor

from sklearn.metrics import r2_score, mean_absolute_error

# Modelos complexos (caixa-preta)
modelos_complexos = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "SVR": SVR(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000),
}

# Surrogates interpretáveis
surrogates = {
    "EBM": ExplainableBoostingRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(max_depth=3, random_state=42),
}

# Pipeline padrão
def make_pipeline_model(model):
    return make_pipeline(TargetEncoder(), StandardScaler(), model)

# Armazenar todos os resultados
resultados = []

for nome_modelo, modelo in modelos_complexos.items():
    print(f"\n🎯 Modelo complexo: {nome_modelo}")

    pipeline_model = make_pipeline_model(modelo)
    pipeline_model.fit(X_train, y_train)
    y_pred_model = pipeline_model.predict(X_train)

    for nome_surrogate, surrogate in surrogates.items():
        pipeline_surrogate = make_pipeline_model(surrogate)
        pipeline_surrogate.fit(X_train, y_pred_model)
        y_pred_surrogate = pipeline_surrogate.predict(X_train)

        r2 = r2_score(y_pred_model, y_pred_surrogate)
        mae = mean_absolute_error(y_pred_model, y_pred_surrogate)

        resultados.append({
            "Modelo_CaixaPreta": nome_modelo,
            "Surrogate": nome_surrogate,
            "R2_surrogate": round(r2, 4),
            "MAE_surrogate": round(mae, 4),
        })

# DataFrame com resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)



🎯 Modelo complexo: Random Forest

🎯 Modelo complexo: XGBoost

🎯 Modelo complexo: SVR

🎯 Modelo complexo: MLP Regressor
   Modelo_CaixaPreta         Surrogate  R2_surrogate  MAE_surrogate
0      Random Forest               EBM        0.8655         0.0447
1      Random Forest  LinearRegression        0.8108         0.0595
2      Random Forest      DecisionTree        0.7500         0.0613
3            XGBoost               EBM        0.7180         0.0736
4            XGBoost  LinearRegression        0.6232         0.0865
5            XGBoost      DecisionTree        0.5319         0.0910
6                SVR               EBM        0.8349         0.0341
7                SVR  LinearRegression        0.6816         0.0537
8                SVR      DecisionTree        0.6908         0.0443
9      MLP Regressor               EBM        0.7444         0.1017
10     MLP Regressor  LinearRegression        0.6635         0.1176
11     MLP Regressor      DecisionTree        0.5665         0.1

In [10]:
# 6. Explicações com EBM
show(ebm.explain_global())  # Importância global (interações e efeitos)


<!-- http://127.0.0.1:7126/139987707415760/ -->